In [11]:
import os.path

if os.path.exists("/content/Image_Data"):
  print("Data exists")
else:
  from google.colab import drive
  drive.mount('/content/drive')
  !unzip "/content/drive/MyDrive/MIT6.819 Project/Image_Data.zip" -d "/content/"
  # !mv "/content/drive/My Drive/MIT6.819 Project/utils.py" "/content/"
  !mv "/content/drive/My Drive/MIT6.819 Project/model_weights.h5" "/content/"


Data exists


In [12]:
import pandas as pd
import numpy as np
import pickle
import re
import os.path
import cv2
       
import sys
import tensorflow as tf
from tensorflow.keras import backend as K
try:
  import tensorflow_addons as tfa
except:
  !pip install tensorflow_addons
  import tensorflow_addons as tfa
  
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import Input, Concatenate, InputLayer, Conv2DTranspose, Resizing
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings 
warnings.filterwarnings('always')
from skimage.transform import warp_polar, warp_coords
try:
  from tensorflow_graphics.math.math_helpers import cartesian_to_spherical_coordinates, spherical_to_cartesian_coordinates
except:
  !pip install --upgrade tensorflow-graphics
  from tensorflow_graphics.math.math_helpers import cartesian_to_spherical_coordinates, spherical_to_cartesian_coordinates

from google.colab import files


# Helper code files
import time
sys.path.append('../')
from utils import get_dataset, set_model_weights, lp_architecture
from IPython.display import clear_output


In [13]:
# Global Variables
input_shape = (224, 224, 3)
num_classes = 1000
batch_size = 1


In [14]:
input_layer, output_layer = lp_architecture()
model = Model(inputs=input_layer, outputs=output_layer, name="LP_MODEL")


In [15]:
model = set_model_weights(model, model_type="custom", 
                          path="/content/model_weights.h5")


In [ ]:
# GRADIENT TAPE

# Algorithm parameters
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
metrics = tf.keras.metrics.Accuracy()
num_epochs = 30

path="/content/Image_Data/"

train_data = get_dataset(batch_size, False, path=path)

# Since batch_size is 1, each epoch will see steps_per_epoch number of images
steps_per_epoch = 128

for i in range(num_epochs):
    print("\nEPOCH:", i+1)
    loss = 0
    
    for bs in range(steps_per_epoch):
        if bs%50==0:
            print("\tBATCH NUMBER:",bs)
        try:
            t1 = train_data.next()
        except:
            print("\t\tFresh Images")
            train_data = get_dataset(batch_size, False, path=path)
            t1 = train_data.next()
        
        with tf.GradientTape() as tape:
            output = model(t1[0])
            loss+= loss_fn(t1[1], output)
            
    gradients = tape.gradient(loss, model.trainable_weights)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))    
    model.save_weights("model_weights.h5")
    print("\t\tUPDATED MODEL WEIGHTS")

    if i%5==0:
      # files.download("model_weights.h5")
      clear_output(wait=True)




EPOCH: 7
	BATCH NUMBER: 0
	BATCH NUMBER: 50
	BATCH NUMBER: 100
		UPDATED MODEL WEIGHTS

EPOCH: 8
	BATCH NUMBER: 0
	BATCH NUMBER: 50
	BATCH NUMBER: 100
		UPDATED MODEL WEIGHTS

EPOCH: 9
	BATCH NUMBER: 0
	BATCH NUMBER: 50


In [ ]:
model.save_weights("model_weights_final.h5")
files.download("model_weights_final.h5")
